In [1]:
from fractions import Fraction

# Given a list of numbers, divide into subsets 
# / {first num, ---} or * {first num, ---}

# {first num, ---} +- rest

# return the list of all expressions with evaluated result

def magimixer(digits):
    # Memoization in python
    a = digits
    a_dict = {}
    for i in range(len(a)):
        a_dict[2 ** i] = a[i]

    add_mem = [None for i in range(2 ** len(a))]
    mul_mem = [None for i in range(2 ** len(a))]
    
    def compute(bitset):
        head_bitset = bitset & -bitset
        if bitset is head_bitset:
            val = a_dict[head_bitset]
            return [(str(val), Fraction(val))]
        else:
            return compute_add(bitset) + compute_mul(bitset)
        
    def compute_add(bitset):
        if add_mem[bitset] is None:
            head_bitset = bitset & -bitset
            head = a_dict[head_bitset]
            if bitset is head_bitset:
                add_mem[bitset] = [(str(head), Fraction(head))]
            else:
                tail_bitset = mask = bitset - head_bitset
                result = []
                while tail_bitset is not 0:
                    # iterate over all possible nonempty combinations of subsets
                    for (expr_head, val_head) in compute_mul(head_bitset):
                        for (expr_tail, val_tail) in compute(tail_bitset):
                            result.append( ("(" + expr_head + "+" + expr_tail + ")", val_head + val_tail) )
                            if val_head >= val_tail:
                                result.append( ("(" + expr_head + "-" + expr_tail + ")", val_head - val_tail) )
                            else:
                                result.append( ("(" + expr_tail + "-" + expr_head + ")", val_tail - val_head) )
                    tail_bitset -= 1
                    tail_bitset &= mask
                    head_bitset = bitset - tail_bitset
                add_mem[bitset] = result
        return add_mem[bitset]
        
    def compute_mul(bitset):
        if mul_mem[bitset] is None:
            head_bitset = bitset & -bitset
            head = a_dict[head_bitset]
            if bitset is head_bitset:
                mul_mem[bitset] = [(str(head), Fraction(head))]
            else:
                tail_bitset = mask = bitset - head_bitset
                result = []
                while tail_bitset is not 0:
                    # iterate over all possible nonempty combinations of subsets
                    for (expr_head, val_head) in compute_add(head_bitset):
                        for (expr_tail, val_tail) in compute(tail_bitset):
                            result.append( ("(" + expr_head + "*" + expr_tail + ")", val_head * val_tail) )
                            if val_tail != Fraction(0, 1) and val_tail != Fraction(1, 1):
                                result.append( ("(" + expr_head + "/" + expr_tail + ")", val_head / val_tail) )
                            if val_head != Fraction(0, 1) and val_head != Fraction(1, 1):
                                result.append( ("(" + expr_tail + "/" + expr_head + ")", val_tail / val_head) )
                    tail_bitset -= 1
                    tail_bitset &= mask
                    head_bitset = bitset - tail_bitset
                mul_mem[bitset] = result
        return mul_mem[bitset]
    
    ans_dict = {}
    for (expr, val) in compute(31):
        if val.denominator == 1:
            if val in ans_dict:
                ans_dict[int(val)].add(expr)
            else:
                ans_dict[int(val)] = {expr}
            
    return ans_dict

In [2]:
for (num, methods) in magimixer((4,4,5,6,6)).items():
    if 11 <= num <= 66 and 1 <= num % 10 <= 6 and len(methods) <= 2:
        print(num, methods)

62 {'((((4*5)-6)*4)+6)', '((4*((4*5)-6))+6)'}


In [3]:
from itertools import combinations_with_replacement

hard_probs = {}

for comb in combinations_with_replacement(range(1, 7), 5):
    for (num, methods) in magimixer(comb).items():
        if 11 <= num <= 66 and 1 <= num % 10 <= 6 and len(methods) <= 2:
            if comb in hard_probs:
                hard_probs[comb].add(num)
            else:
                hard_probs[comb] = {num}

In [4]:
prob_list = list(hard_probs.items())

In [5]:
prob_list = []
for (comb, values) in hard_probs.items():
    for value in values:
        prob_list.append( (comb, value) )

In [6]:
from random import choice

In [7]:
prob = choice(prob_list)

In [8]:
print(prob)

((1, 1, 2, 6, 6), 46)


In [9]:
magimixer(prob[0])[prob[1]]

{'((((2+6)*6)-1)-1)', '(((1+(1+6))*6)-2)'}